In [6]:
import os
import json
import boto3
import pandas as pd
import numpy as np
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer
from boto3.dynamodb.conditions import Key
from decimal import Decimal
from dotenv import load_dotenv
from datetime import datetime
import json

load_dotenv()

True

In [7]:

ts= TypeSerializer()
td = TypeDeserializer()

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

In [8]:
def query_sql(beer_id, dynamodb=None):
    if not dynamodb:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))
    stmt = "SELECT BREW_ID,PI_TMSTP,STAGE,location,temperature,thermo_loc,thermo_temp FROM BREW_MON WHERE BREW_ID='" + beer_id + "'  ORDER BY PI_TMSTP DESC"
   # print(lambda_handler(event=brew1))
    response = dyna_clnt.execute_statement(Statement=stmt)
    #print("HI")
    return response    
 

In [9]:
lines = query_sql(beer_id='SCWG_IPA')
df_raw = pd.json_normalize(lines, record_path =['Items'])

In [10]:
    df_raw['pi_time']=pd.to_datetime(df_raw['PI_TMSTP.S'])
    df_raw['temperature_n']=pd.to_numeric(df_raw['temperature.S'], downcast="float")
    df_raw['thermo_temp_n']=pd.to_numeric(df_raw['thermo_temp.N'], downcast="float")    
    df_raw.set_index('pi_time',inplace=True)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n > 50].index)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n < 0].index)
    df_raw = df_raw.drop(columns=['BREW_ID.S','PI_TMSTP.S','STAGE.S','location.S','thermo_loc.S','temperature.S','thermo_temp.N'])


In [29]:
    df_4h = df_raw.groupby([pd.Grouper(freq='2H', level='pi_time')]).mean(numeric_only=True)
    df_4h = df_4h[df_4h['temperature_n'].notna()]
    df_4h = df_4h[df_4h['thermo_temp_n'].notna()]
    df_4h['temperature_n'] = df_4h['temperature_n'].astype('float64').round(decimals=0)
    df_4h['thermo_temp_n'] = df_4h['thermo_temp_n'].astype('float64').round(decimals=0)
    df_4h = df_4h.drop(columns=['pi_time'])

KeyError: "['pi_time'] not found in axis"

In [ ]:
print(df_4h)

In [13]:
    df_4h=df_4h.reset_index()
  

In [ ]:
    json_4h = {'Items': df_4h_json}

In [17]:
print(new_items)


[{'pi_time': Timestamp('2021-09-08 00:00:00'), 'temperature_n': 13.0, 'thermo_temp_n': 18.0}, {'pi_time': Timestamp('2021-09-09 22:00:00'), 'temperature_n': 20.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 00:00:00'), 'temperature_n': 18.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 02:00:00'), 'temperature_n': 18.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 04:00:00'), 'temperature_n': 17.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 06:00:00'), 'temperature_n': 17.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 08:00:00'), 'temperature_n': 19.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 10:00:00'), 'temperature_n': 21.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 12:00:00'), 'temperature_n': 22.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 14:00:00'), 'temperature_n': 23.0, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 16:00:00'), 'temperature_n': 23.0, 

In [18]:

new_items = df_4h.to_dict(orient='records')
changed_data = json.loads(json.dumps(new_items), parse_float=Decimal, parse_timestamp=str)


TypeError: Object of type Timestamp is not JSON serializable

In [ ]:
print(new_items)

In [27]:
# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))

table = dynamodb.Table('BREW_LOG')

In [20]:
data2 = {
    'temperature': [60.3, 62, 65, 63, 61],
    'timestamp': [
        '2023-12-18 00:00:00',
        '2023-12-18 02:00:00',
        '2023-12-18 04:00:00',
        '2023-12-18 06:00:00',
        '2023-12-18 08:00:00'
    ]
}
df_tss = pd.DataFrame(data2)
#print(df_tss)
items_from_test = df_tss.to_dict(orient='records')


#df = pd.DataFrame(changed_data)



# Convert DataFrame to a list of dictionaries
#items_from_test = df.to_dict(orient='records',parse_float=Decimal)
#items_from_test = pd.io.json.dumps(df, double_precision=2)

In [23]:
print(data2)
#print(changed_data)
#print(df)
print(items_from_test)

{'temperature': [60.3, 62, 65, 63, 61], 'timestamp': ['2023-12-18 00:00:00', '2023-12-18 02:00:00', '2023-12-18 04:00:00', '2023-12-18 06:00:00', '2023-12-18 08:00:00']}
[{'temperature': 60.3, 'timestamp': '2023-12-18 00:00:00'}, {'temperature': 62.0, 'timestamp': '2023-12-18 02:00:00'}, {'temperature': 65.0, 'timestamp': '2023-12-18 04:00:00'}, {'temperature': 63.0, 'timestamp': '2023-12-18 06:00:00'}, {'temperature': 61.0, 'timestamp': '2023-12-18 08:00:00'}]


In [24]:
changed_data = json.loads(json.dumps(items_from_test), parse_float=Decimal)

In [25]:
print(changed_data)

[{'temperature': Decimal('60.3'), 'timestamp': '2023-12-18 00:00:00'}, {'temperature': Decimal('62.0'), 'timestamp': '2023-12-18 02:00:00'}, {'temperature': Decimal('65.0'), 'timestamp': '2023-12-18 04:00:00'}, {'temperature': Decimal('63.0'), 'timestamp': '2023-12-18 06:00:00'}, {'temperature': Decimal('61.0'), 'timestamp': '2023-12-18 08:00:00'}]


In [ ]:
print(df_tss)
print('-----------------------------')
print(df_4h)

In [28]:
table.update_item(
   Key={"BEER_ID":"SCWG_IPA", "BEER_DATE": "2021-09-05"},
                UpdateExpression="set TEMP_DATA_TEST233=:r",
                ExpressionAttributeValues={":r": changed_data},
                ReturnValues="UPDATED_NEW"
)

{'Attributes': {'TEMP_DATA_TEST233': [{'temperature': Decimal('60.3'),
    'timestamp': '2023-12-18 00:00:00'},
   {'temperature': Decimal('62'), 'timestamp': '2023-12-18 02:00:00'},
   {'temperature': Decimal('65'), 'timestamp': '2023-12-18 04:00:00'},
   {'temperature': Decimal('63'), 'timestamp': '2023-12-18 06:00:00'},
   {'temperature': Decimal('61'), 'timestamp': '2023-12-18 08:00:00'}]},
 'ResponseMetadata': {'RequestId': 'MR4UIEMUVF9F2VRIRGE9NQN5GJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 20 Dec 2023 17:22:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '411',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MR4UIEMUVF9F2VRIRGE9NQN5GJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '793604352'},
  'RetryAttempts': 0}}

In [ ]:
print(new_items)

In [ ]:
items_from_raw=json.loads(json.dumps(new_items), parse_float=Decimal)

In [ ]:
table.update_item(
   Key={"BEER_ID":"SCWG_IPA", "BEER_DATE": "2021-09-05"},
                UpdateExpression="set TEMP_DATA_RAW=:r",
                ExpressionAttributeValues={":r": items_from_raw},
                ReturnValues="UPDATED_NEW"
)

In [ ]:
print(items_from_test)
print('0----------------------------------------------')
print(items_from_raw)